In [8]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [9]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
# Preprocess the data 
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [11]:
# Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [13]:
# One hot encode 'Geography'
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [14]:
# Combine one hot encoded columns with orignal data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
# Split the data into features and target
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [16]:
# Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Scale these features  
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.fit_transform(X_test)

In [18]:
# Save the encoders and scalar for later use 
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scalar.pkl', 'wb') as file:
    pickle.dump(scalar, file)


### ANN with regression problem statement

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [21]:
# build the model 
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) # Output layer for regression 
])

# Compile the model 
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up tensorboard
log_dir = 'regressionlogs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [24]:
# Set up early stopping 
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [25]:
# Train the model 
history = model.fit(
    X_train, y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 4s 4ms/step - loss: 100370.3047 - mae: 100370.3047 - val_loss: 98489.6641 - val_mae: 98489.6641
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 99508.2422 - mae: 99508.2422 - val_loss: 96722.2500 - val_mae: 96722.2500
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 96404.5625 - mae: 96404.5625 - val_loss: 92175.0938 - val_mae: 92175.0938
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 90339.3750 - mae: 90339.3750 - val_loss: 84734.6641 - val_mae: 84734.6641
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 81682.2812 - mae: 81682.2812 - val_loss: 75478.3281 - val_mae: 75478.3281
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 71985.6719 - mae: 71985.6719 - val_loss: 66316.7188 - val_mae: 66316.7188
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 63181.6406 - mae: 63181.640

In [26]:
%load_ext tensorboard

In [28]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 16448), started 0:02:03 ago. (Use '!kill 16448' to kill it.)

In [29]:
# Evaluate model for test data 
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

63/63 [==============================] - 1s 7ms/step - loss: 50370.9844 - mae: 50370.9844
Test MAE: 50370.984375


In [30]:
model.save('regression_model.h5')

c:\Users\Shaunak\GenAI with LangChain and HuggingFace\End-to-end ANN project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
